In [1]:
import random
import networkx as nx
import numpy as np
import karateclub
from tkinter import _flatten
import community

In [19]:
import MGCF

### 1. Generate $K$ and $M$

In [101]:
K = random.randint(1, 10)
M = random.randint(1, 100)
print(K)

10


### 2. Gnerate Nodes

In [102]:
average_cluster_size = random.randint(20, 40)

center_x_list = []
center_y_list = []
for i in range(K):
    center_x_list.append(round(10*random.random(), 4))
    center_y_list.append(round(10*random.random(), 4))
node_groundtruth = {}
node_location = {}
idx = 0
for i in range(K):
    for j in range(average_cluster_size):
        node_groundtruth[idx] = i
        node_location[idx] = [center_x_list[i] + round(random.gauss(0, 1), 4), center_y_list[i] + round(random.gauss(0, 1), 4)]
        idx = idx + 1
N = len(node_groundtruth)
print('节点总数', N)
G = nx.Graph()
G.add_nodes_from(list(range(N)))

ground_truth = {}
for k, v in node_groundtruth.items():
    if v not in ground_truth:
        ground_truth[v] = [k]
    else:
        ground_truth[v].append(k)

节点总数 360


### 3. k-nearest neighborhood

In [103]:
loc_np = np.array(list(node_location.values()))
x_np = loc_np[:, 0].reshape((1, -1))
y_np = loc_np[:, 1].reshape((1, -1))
distance_np = np.square(x_np - x_np.T)+np.square(y_np - y_np.T)
k_nearest = {}
k = int(average_cluster_size/2)
for i in range(N):
    k_near_pre = list(np.argsort(distance_np[i])[:k+1:])
    k_near_pre.remove(i)
    k_nearest[i] = k_near_pre
for i in range(N):
    for j in k_nearest[i]:
        G.add_edge(i, j, weight = np.exp(-1/2*distance_np[i, j]))
        
G.to_undirected()

### 4. Generate $f_{k, m}$

In [104]:
omega = 1/K*np.ones(K)
beta = 1/M*np.ones(M)
F = omega.reshape((1, -1))*beta.reshape((-1, 1))
F = F*K*M
# F [M, K]
print(average_cluster_size)

36


### 5. Allocate for $X$

In [105]:
X = np.zeros((N, M))
for i in range(M):
    for j in range(K):
        tmp = np.random.dirichlet(np.ones(average_cluster_size), size=1)*F[i, j]
        X[j*average_cluster_size:(j+1)*average_cluster_size:, i] = tmp

### 6. for Algorithm Process & Evaluation

In [106]:
# 1. DANMF

model1 = karateclub.DANMF()
model1.fit(G)
cluster_membership = model1.get_memberships()
clusters = cluster_membership
labels_pred = list(_flatten(list(clusters.values())))
partition = dict(zip(list(range(len(labels_pred))), labels_pred))
partition_2 = {}
for i in range(len(labels_pred)):
    if labels_pred[i] not in partition_2:
        partition_2[labels_pred[i]] = [i]
    else:
        partition_2[labels_pred[i]].append(i)
        
print(round(fscore(partition_2, ground_truth), 3))
print(round(nmi(partition_2, ground_truth), 3))
print(round(jaccard_similarity(partition_2, ground_truth), 3))

0.485
0.496
0.328


In [107]:
# 2. NNSED

model1 = karateclub.NNSED(dimensions=K)
model1.fit(G)
cluster_membership = model1.get_memberships()
clusters = cluster_membership
labels_pred = list(_flatten(list(clusters.values())))
partition = dict(zip(list(range(len(labels_pred))), labels_pred))
partition_2 = {}
for i in range(len(labels_pred)):
    if labels_pred[i] not in partition_2:
        partition_2[labels_pred[i]] = [i]
    else:
        partition_2[labels_pred[i]].append(i)
        
print(round(fscore(partition_2, ground_truth), 3))
print(round(nmi(partition_2, ground_truth), 3))
print(round(jaccard_similarity(partition_2, ground_truth), 3))

0.42
0.309
0.273


In [108]:
# 3. MNMF

model1 = karateclub.MNMF(clusters=K)
model1.fit(G)
cluster_membership = model1.get_memberships()
clusters = cluster_membership
labels_pred = list(_flatten(list(clusters.values())))
partition = dict(zip(list(range(len(labels_pred))), labels_pred))
partition_2 = {}
for i in range(len(labels_pred)):
    if labels_pred[i] not in partition_2:
        partition_2[labels_pred[i]] = [i]
    else:
        partition_2[labels_pred[i]].append(i)
        
print(round(fscore(partition_2, ground_truth), 3))
print(round(nmi(partition_2, ground_truth), 3))
print(round(jaccard_similarity(partition_2, ground_truth), 3))

0.504
0.484
0.361


In [109]:
# 4. BigClam

model1 = karateclub.BigClam(dimensions=K)
model1.fit(G)
cluster_membership = model1.get_memberships()
clusters = cluster_membership
labels_pred = list(_flatten(list(clusters.values())))
partition = dict(zip(list(range(len(labels_pred))), labels_pred))
partition_2 = {}
for i in range(len(labels_pred)):
    if labels_pred[i] not in partition_2:
        partition_2[labels_pred[i]] = [i]
    else:
        partition_2[labels_pred[i]].append(i)
        
print(round(fscore(partition_2, ground_truth), 3))
print(round(nmi(partition_2, ground_truth), 3))
print(round(jaccard_similarity(partition_2, ground_truth), 3))

0.383
0.263
0.245


In [110]:
# 5. SNMF

model1 = karateclub.SymmNMF(dimensions=K)
model1.fit(G)
cluster_membership = model1.get_memberships()
clusters = cluster_membership
labels_pred = list(_flatten(list(clusters.values())))
partition = dict(zip(list(range(len(labels_pred))), labels_pred))
partition_2 = {}
for i in range(len(labels_pred)):
    if labels_pred[i] not in partition_2:
        partition_2[labels_pred[i]] = [i]
    else:
        partition_2[labels_pred[i]].append(i)
        
print(round(fscore(partition_2, ground_truth), 3))
print(round(nmi(partition_2, ground_truth), 3))
print(round(jaccard_similarity(partition_2, ground_truth), 3))

0.238
0.138
0.137


In [111]:
# 6. GEMSEC

model1 = karateclub.GEMSEC(dimensions=K)
model1.fit(G)
cluster_membership = model1.get_memberships()
clusters = cluster_membership
labels_pred = list(_flatten(list(clusters.values())))
partition = dict(zip(list(range(len(labels_pred))), labels_pred))
partition_2 = {}
for i in range(len(labels_pred)):
    if labels_pred[i] not in partition_2:
        partition_2[labels_pred[i]] = [i]
    else:
        partition_2[labels_pred[i]].append(i)
        
print(round(fscore(partition_2, ground_truth), 3))
print(round(nmi(partition_2, ground_truth), 3))
print(round(jaccard_similarity(partition_2, ground_truth), 3))

0.538
0.506
0.395


In [112]:
# 7. SCD

model1 = karateclub.SCD()
model1.fit(G)
cluster_membership = model1.get_memberships()
clusters = cluster_membership
labels_pred = list(_flatten(list(clusters.values())))
partition = dict(zip(list(range(len(labels_pred))), labels_pred))
partition_2 = {}
for i in range(len(labels_pred)):
    if labels_pred[i] not in partition_2:
        partition_2[labels_pred[i]] = [i]
    else:
        partition_2[labels_pred[i]].append(i)
        
print(round(fscore(partition_2, ground_truth), 3))
print(round(nmi(partition_2, ground_truth), 3))
print(round(jaccard_similarity(partition_2, ground_truth), 3))

0.401
0.532
0.273


In [113]:
# 8. LPA

model1 = karateclub.LabelPropagation(seed=10, iterations=100)
model1.fit(G)
cluster_membership = model1.get_memberships()
clusters = cluster_membership
labels_pred = list(_flatten(list(clusters.values())))
partition = dict(zip(list(range(len(labels_pred))), labels_pred))
partition_2 = {}
for i in range(len(labels_pred)):
    if labels_pred[i] not in partition_2:
        partition_2[labels_pred[i]] = [i]
    else:
        partition_2[labels_pred[i]].append(i)
        
print(round(fscore(partition_2, ground_truth), 3))
print(round(nmi(partition_2, ground_truth), 3))
print(round(jaccard_similarity(partition_2, ground_truth), 3))

0.172
0.049
0.094


In [114]:
# 9. Louvain

partition = community.best_partition(G, weight='weight')
labels_pred = partition
partition_2 = {}
for i in range(len(labels_pred)):
    if labels_pred[i] not in partition_2:
        partition_2[labels_pred[i]] = [i]
    else:
        partition_2[labels_pred[i]].append(i)
        
print(round(fscore(partition_2, ground_truth), 3))
print(round(nmi(partition_2, ground_truth), 3))
print(round(jaccard_similarity(partition_2, ground_truth), 3))

0.573
0.527
0.434


In [117]:
# Ours
partition = MGCF.main(clusters_number=K, feature=X, multi_graph=[G], alpha_1=1, alpha_2=0.1, alpha_3=0.01, weight_list=[1], alpha=2, epoch_num=250)

Epoch:  0 ; soft_mod:  0.003017 ; loss_cons:  0.104492 ; loss_clus:  0.0
Epoch:  10 ; soft_mod:  0.006028 ; loss_cons:  0.064328 ; loss_clus:  0.962183
Epoch:  20 ; soft_mod:  0.034588 ; loss_cons:  0.069634 ; loss_clus:  1.078709
Epoch:  30 ; soft_mod:  0.051057 ; loss_cons:  0.06526 ; loss_clus:  0.959512
Epoch:  40 ; soft_mod:  0.066604 ; loss_cons:  0.051826 ; loss_clus:  0.900025
Epoch:  50 ; soft_mod:  0.141815 ; loss_cons:  0.042637 ; loss_clus:  0.840196
Epoch:  60 ; soft_mod:  0.285952 ; loss_cons:  0.049788 ; loss_clus:  0.849018
Epoch:  70 ; soft_mod:  0.366723 ; loss_cons:  0.045985 ; loss_clus:  0.857807
Epoch:  80 ; soft_mod:  0.476688 ; loss_cons:  0.029378 ; loss_clus:  0.900582
Epoch:  90 ; soft_mod:  0.545812 ; loss_cons:  0.037601 ; loss_clus:  1.006857
Epoch:  100 ; soft_mod:  0.613745 ; loss_cons:  0.029818 ; loss_clus:  1.198673
Epoch:  110 ; soft_mod:  0.645755 ; loss_cons:  0.034505 ; loss_clus:  1.476215
Epoch:  120 ; soft_mod:  0.681986 ; loss_cons:  0.037807 

In [118]:
labels_pred = partition
partition_2 = {}
for i in range(len(labels_pred)):
    if labels_pred[i] not in partition_2:
        partition_2[labels_pred[i]] = [i]
    else:
        partition_2[labels_pred[i]].append(i)
        
print(round(fscore(partition_2, ground_truth), 3))
print(round(nmi(partition_2, ground_truth), 3))
print(round(jaccard_similarity(partition_2, ground_truth), 3))

0.48
0.541
0.364


### 7. Evaluation Functions

In [9]:
# partition: cluster -- node_list
# ground_truth: node -- cluster

def nmi(partition, ground_truth):
    node_num = 0
    for k, v in partition.items():
        node_num += len(v)

    hx = 0
    for k, v in partition.items():
        p = len(v)/node_num
        hx += p*np.log2(1/(p+0.0000001))

    hy = 0
    for k, v in ground_truth.items():
        p = len(v)/node_num
        hy += p*np.log2(1/(p+0.0000001))

    ixy = 0
    for k1, v1 in partition.items():
        for k2, v2 in ground_truth.items():
            pxy = len(set(v1) & set(v2))/node_num
            px = len(v1)/node_num
            py = len(v2)/node_num
            ixy += pxy*np.log2((pxy+0.0000001)/(px+0.0000001)/(py+0.0000001))

    return ixy*2/(hx+hy)


def fscore(partition, ground_truth):
    F = 0
    count = 0
    for k1, v1 in partition.items():
        f = 0
        s1 = set(v1)
        for k2, v2 in ground_truth.items():
            s2 = set(v2)
            intersection = s1 & s2
            precision = len(intersection)/len(s1)
            recall = len(intersection)/len(s2)
            if precision*recall != 0:
                tmp = 2*precision*recall/(precision + recall)
                if tmp > f: f = tmp
            else:
                continue
        F += f
        count += 1
    F = F/count
    return F


def jaccard_similarity(partition, ground_truth):
    JS = 0
    count = 0
    for k1, v1 in partition.items():
        js = 0
        s1 = set(v1)
        for k2, v2 in ground_truth.items():
            s2 = set(v2)
            tmp = len(s1 & s2)/len(s1 | s2)
            if tmp>js: js = tmp
        JS += js
        count += 1
    JS = JS/count
    return JS